## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-10-23-49-13 +0000,nypost,Hakeem Jeffries’ team blasts Zohran Mamdani’s ...,https://nypost.com/2025/07/10/us-news/hakeem-j...
1,2025-07-10-23-33-41 +0000,nyt,At Least 13 People Died by Suicide Amid U.K. P...,https://www.nytimes.com/2025/07/10/world/europ...
2,2025-07-10-23-29-11 +0000,nyt,3 Teenagers Arrested Over Cyberattacks That Co...,https://www.nytimes.com/2025/07/10/business/cy...
3,2025-07-10-23-23-47 +0000,bbc,Mahmoud Khalil files $20m claim against Trump ...,https://www.bbc.com/news/articles/cvg4188gl4zo
4,2025-07-10-23-05-30 +0000,bbc,"'Autofocus' specs promise sharp vision, near o...",https://www.bbc.com/news/articles/cj6r06d7xdjo


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2345/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
33,trump,26
142,brazil,6
312,new,5
143,tariffs,5
50,secret,4


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
35,2025-07-10-17-43-26 +0000,nypost,Pictured: Trump’s Secret Service agent who was...,https://nypost.com/2025/07/10/us-news/secret-s...,53
25,2025-07-10-19-21-22 +0000,nypost,Senate clears path to confirm first judge of T...,https://nypost.com/2025/07/10/us-news/senate-c...,52
46,2025-07-10-15-01-21 +0000,nypost,Marco Rubio slams Russia over Ukrainian war de...,https://nypost.com/2025/07/10/us-news/marco-ru...,52
26,2025-07-10-18-53-57 +0000,nypost,Trump reflects on assassination attempt one ye...,https://nypost.com/2025/07/10/us-news/trump-re...,52
63,2025-07-10-12-04-11 +0000,nyt,"Trump Pledges 50% Tariffs Against Brazil, Citi...",https://www.nytimes.com/2025/07/09/world/ameri...,51


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
35,53,2025-07-10-17-43-26 +0000,nypost,Pictured: Trump’s Secret Service agent who was...,https://nypost.com/2025/07/10/us-news/secret-s...
88,28,2025-07-10-04-39-28 +0000,nypost,Ex-CIA Director John Brennan says he’s had ‘no...,https://nypost.com/2025/07/10/us-news/ex-cia-d...
25,26,2025-07-10-19-21-22 +0000,nypost,Senate clears path to confirm first judge of T...,https://nypost.com/2025/07/10/us-news/senate-c...
46,26,2025-07-10-15-01-21 +0000,nypost,Marco Rubio slams Russia over Ukrainian war de...,https://nypost.com/2025/07/10/us-news/marco-ru...
63,25,2025-07-10-12-04-11 +0000,nyt,"Trump Pledges 50% Tariffs Against Brazil, Citi...",https://www.nytimes.com/2025/07/09/world/ameri...
0,21,2025-07-10-23-49-13 +0000,nypost,Hakeem Jeffries’ team blasts Zohran Mamdani’s ...,https://nypost.com/2025/07/10/us-news/hakeem-j...
40,20,2025-07-10-16-08-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...
95,18,2025-07-10-01-20-22 +0000,nypost,Supreme Court blocks Florida law banning illeg...,https://nypost.com/2025/07/09/us-news/supreme-...
91,18,2025-07-10-03-11-18 +0000,cbc,New Mexico residents watch in shock as flash f...,https://www.cbc.ca/news/world/house-swept-away...
96,17,2025-07-10-01-17-00 +0000,wsj,A brewing debate inside the Federal Reserve ov...,https://www.wsj.com/economy/central-banking/wh...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
